<a href="https://colab.research.google.com/github/marek-bardonski/airev-advanced-workshops/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AI REV LLC - Copyrights 2020
# [Google Colab] Change runtime mode to GPU
# https://docs.rapids.ai/

!nvcc --version
!pip3 install wget

# https://github.com/rapidsai/cudf/issues/3390
!pip3 install pyarrow==0.15.0 ## Workaround to cover up for Google Colab bug

#!pip3 install cudf-cuda100
#!pip3 install nvstrings-cuda100
#!pip3 install cuml-cuda100
#!pip3 install nvvm-cuda100

# RAPIDS installation script. Thanks Ritchie Ng and NVIDIA Corporation.
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
--2020-01-02 21:09:01--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745

**Remember to restart the runtime at this moment and restart the procedure from the top.** 



In [0]:
# set environment vars
import sys, os, shutil

sys.path.append('/usr/local/lib/python3.6/site-packages/')
sys.path.append('/usr/local/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

import wget
from zipfile import ZipFile
import os
import cudf
import sys, os
import nvcategory
import os
import numpy as np
#import cuml (bug https://gitmemory.com/issue/rapidsai/cuml/404/477551898)
import nvstrings
import nltk
from numba import cuda
import json
import nvtext
import ctypes
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
torch.cuda.is_available()

**If you got import errors, please look above or ask instructor.**

In [3]:
# Raw datasets are samples from WebHose.io 
print('Beginning dataset download with wget module')

url = 'https://bardonski.pl/chineese.zip'
wget.download(url)
url = 'https://bardonski.pl/arabic.zip'
wget.download(url)
url = 'https://bardonski.pl/arabic-true-pr.csv'
wget.download(url)

print('Beginning word vectors download with wget module')
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz'
print('Word vectors unzip')
wget.download(url)
!gzip -d cc.ar.300.vec.gz

Beginning dataset download with wget module
Beginning word vectors download with wget module
Word vectors unzip


In [0]:
zip = ZipFile('chineese.zip')
zip.extractall()

!mkdir chineese

zip = ZipFile('630_webhose-2016-10_20170904084325.zip')
zip.extractall('chineese')

In [0]:
zip = ZipFile('arabic.zip')
zip.extractall()

!mkdir arabic

zip = ZipFile('627_webhose-2016-10_20170904083346.zip')
zip.extractall('arabic')



In [6]:
#Expected 236384 arabic articles
#Expected 316004 chineese articles
!ls arabic -l | wc -l
!ls chineese -l | wc -l


236384
316004


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# Thanks VibhuJawa
def get_text(lines):
    """
        returns non empty lines from a list of lines
    """
    decoded = json.loads(lines[0])
    clean_lines = decoded['text']
    return [clean_lines]

def get_txt_lines(data_dir):
    """
        Read text lines from gutenberg tests
        returns (text_ls,fname_ls) where 
        text_ls = input_text_lines and fname_ls = list of file names
    """
    text_ls = []
    fname_ls = []
    for fn in os.listdir(data_dir):
        full_fn = os.path.join(data_dir,fn)
        with open(full_fn,encoding="utf-8",errors="ignore") as f:
            content = f.readlines()
            content = get_text(content)
            if content is not None:
                text_ls += content
                ### dont add .txt to the file
                fname_ls += [fn[:-4]]*len(content)
        #return text_ls, fname_ls    
    
    return text_ls, fname_ls    
    
print("File Read Time:")
%time txt_ls,fname_ls = get_txt_lines('arabic')
df = cudf.DataFrame()

print("\nCUDF  Creation Time:")
%time df['text'] = nvstrings.to_device(txt_ls)

File Read Time:
CPU times: user 12.3 s, sys: 2.7 s, total: 15 s
Wall time: 15.1 s

CUDF  Creation Time:
CPU times: user 2.43 s, sys: 876 ms, total: 3.31 s
Wall time: 3.39 s


In [9]:
print("Number of lines in the DF = {:,}".format(len(df)))

Number of lines in the DF = 236,383


In [10]:
df.head(10).to_pandas()

,text
0,أعربت الأمم المتحدة عن «قلقها البالغ» جراء أمن...
1,كتب رامى ناجى \nهنأ المهندس خالد عبد العزيز وز...
2,تعاني الكثيرات من مشكلة تساقط الشعر وتقصفه، وا...
3,السفير الكويتي يستقبل مسؤولي جمعية إطعام \nا...
4,حث أحمد مرتضى منصور، عضو مجلس الزمالك، والمشرف...
5,19 أكتوبر 2016-12:11 pm محاكمة محمد مرسي \nقضت...
6,كتبت زينب عبد المنعم يعد مساعد جوجل واحدا من أ...
7,إصابة مجند برصاص إرهابيين في اشتباكات بجنوب ال...
8,الإسكندرية – أسماء على بدر عقدت كلية العلوم جا...
9,رفض طلب رد قاضى أحداث الإرشاد وتغريم البلتاجى ...


In [11]:
STOPWORDS = nltk.corpus.stopwords.words('arabic')

filters = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\~', '\t','\\n',"'",",",'~' , '—']

def preprocess_text(input_strs , filters=None , stopwords=STOPWORDS):
    """
        * filter punctuation
        * to_lower
        * remove stop words (from nltk corpus)
        * remove multiple spaces with one
        * remove leading spaces    
    """
    
    # filter punctuation and case conversion
    input_strs = input_strs.str.replace_multi(filters, ' ', regex=False)
    input_strs = input_strs.str.lower()
        
    # remove stopwords
    stopwords_gpu = nvstrings.to_device(stopwords)
    input_strs = nvtext.replace_tokens(input_strs.data, stopwords_gpu, ' ')
    input_strs = cudf.Series(input_strs)
        
    # replace multiple spaces with single one and strip leading/trailing spaces
    input_strs = input_strs.str.replace(r"\s+", ' ', regex=True)
    input_strs = input_strs.str.strip(' ')
    
    return input_strs

def preprocess_text_df(df, text_cols=['text'], **kwargs):
    for col in text_cols:
        df[col] = preprocess_text(df[col], **kwargs)
    return  df

%time df = preprocess_text_df(df, filters=filters)

CPU times: user 2min 52s, sys: 2min 4s, total: 4min 57s
Wall time: 4min 57s


In [17]:
df.head(5).to_pandas()

,text
0,أعربت الأمم المتحدة «قلقها البالغ» جراء أمن يص...
1,كتب رامى ناجى هنأ المهندس خالد عبد العزيز وزير...
2,تعاني الكثيرات مشكلة تساقط الشعر وتقصفه، يكون ...
3,السفير الكويتي يستقبل مسؤولي جمعية إطعام الريا...
4,حث أحمد مرتضى منصور، عضو مجلس الزمالك، والمشرف...


In [0]:
SAMPLE_SIZE = 10000
df2 = df.head(SAMPLE_SIZE)

In [14]:
df2.to_pandas()

,text
0,أعربت الأمم المتحدة «قلقها البالغ» جراء أمن يص...
1,كتب رامى ناجى هنأ المهندس خالد عبد العزيز وزير...
2,تعاني الكثيرات مشكلة تساقط الشعر وتقصفه، يكون ...
3,السفير الكويتي يستقبل مسؤولي جمعية إطعام الريا...
4,حث أحمد مرتضى منصور، عضو مجلس الزمالك، والمشرف...
...,...
9995,ترامب يرحب بقرار إف بى آى بالتحقيق فى رسائل إل...
9996,حجم الخط ع ع ع قال الدكتور خالد حنفي، وزير الت...
9997,وقع انفجار عنيف قرب مركز للشرطة حي يني بوسنة ب...
9998,0 57 تحتفظ اليابان بمواقف لمئات الدراجات تحت ا...


In [15]:
sum(df2['text'].str.find('بيتكوين'))

-9959

In [0]:
# TODO - Remove the articles containing word 'bitcoin' (optional)


In [16]:
pre_df = cudf.read_csv("cc.ar.300.vec",
                       header=None,
                       delim_whitespace=True,
                       quoting=3,
                       skiprows=1)  #ignore quoting
print(pre_df.head())

   0       1       2       3       4  ...     296     297     298     299     300
0  -  0.0345  0.0229  0.0243 -0.2402  ... -0.0063  0.2203 -0.0396  0.0080  0.0263
1  |  0.1051  0.0187  0.1182 -0.9800  ... -0.0666  0.2379  0.0040  0.0112 -0.0510
2  . -0.1151  0.0741  0.0628 -0.0505  ...  0.0303  0.0113  0.0752 -0.0260  0.0481
3  : -0.1079  0.0184  0.0691 -0.0130  ...  0.0270  0.1038 -0.0907 -0.0440  0.0687
4  ، -0.0181 -0.0114 -0.0830  0.0327  ... -0.0078 -0.0752  0.0450  0.0654 -0.0287

[5 rows x 301 columns]


In [18]:
# Read the file with cudf
names = ['query', 'title', 'text', 'link', 'desc','other']
# Note 'int' for 3rd column- text will be hashed
dtypes = ['str', 'str', 'str', 'str', 'str', 'str']
df_pos = cudf.read_csv('arabic-true-pr.csv', delimiter=',',
                   names=names, dtype=dtypes,
                   skiprows=1)
df_pos.head(15).to_pandas()

,query,title,text,link,desc,other
0,None,ارتفاع سعر البيتكوين يدفع سوق العملات الرقمية ...,\nارتفع سعر البيتكوين من 8900 دولار إلى 11000 ...,https://arabfolio.com/2019/06/23/ارتفاع-سعر-ال...,None,
1,البيتكوين ينمو,356 ألف دولار ، هذا هو سعر البيتكوين عام 2022,لا يزال سعر عملة البيتكوين على المسار الصحيح ل...,https://arabfolio.com/2019/07/27/356-ألف-دولار...,Title,
2,ar.coinnewstelegraph.com,توقف عملات Bitcoin مؤقتًا ولكن ليس من المرجح أ...,"ارتفع سعر البيتكوين مؤخرًا فوق 7,300 دولار قبل...",https://ar.coinnewstelegraph.com/bitcoin-rally...,There is a major bullish trend line forming wi...,
3,ar.coinnewstelegraph.com,قد يرتفع سعر البيتكوين إلى 9 آلاف دولار قبل ال...,"مع توجه Bitcoin حاليًا إلى نطاق يتراوح بين 7,0...",https://ar.coinnewstelegraph.com/bitcoin-price...,Title,
4,البيتكوين ينمو,أذكى الأصول لدخول السوق: سوف تنمو Bitcoin ببطء...,ظل مالكو عملة البيتكوين والعملات المشفرة ينتظر...,https://ar.0xzx.com/2019070715832.html,Title,
5,البيتكوين ينمو,تعلم كيف يجب أن تنمو Bitcoin حتى يفوز McAfee ب...,في عام 2017 ، راهن جون مكافي على قمم بيتكوين [...,https://ar.0xzx.com/2019122355788.html,Tekst,
6,البيتكوين ينمو,توقعات بيتكوين 2019 | 2020 | 2025 | 2030: توقع...,2018 هو عام مجنون بالنسبة لأسعار العملة المشفر...,https://ar.0xzx.com/2019111846070.html,Tekst,
7,البيتكوين ينمو,مستخدمو بيتكوين يكدسون مقتنياتهم بينما تُظهر ا...,تام شمس - الاثنين 30 ديسمبر 2019 12:54 صباحاً ...,https://gulf365.co/cryptocurrency-news/6481754...,Tekst,
8,البيتكوين ينمو,توقعات البيتكوين 2020,بقلم: مايكل كوشار\r\n\r\nتقترب البتكوين من نها...,https://arab.dailyforex.com/forex-articles/201...,None,
9,البيتكوين ينمو,BTC محلل الخطوط العريضة للكسور الصاعد التنبؤ ا...,تحتوي حركة أسعار BTC الحالية على عدد من أوجه ا...,https://coinrevolution.com/ar/btc-%D9%85%D8%AD...,None,


In [0]:
df_pos = df_pos.drop(['query', 'title', 'link', 'desc', 'other'])

In [0]:
df_pos.add_column('target', 1)

In [0]:
df_pos = df_pos.dropna()

In [24]:
df2.add_column('target', 0)
df2

,text,target
0,أعربت الأمم المتحدة «قلقها البالغ» جراء أمن يص...,0
1,كتب رامى ناجى هنأ المهندس خالد عبد العزيز وزير...,0
2,تعاني الكثيرات مشكلة تساقط الشعر وتقصفه، يكون ...,0
3,السفير الكويتي يستقبل مسؤولي جمعية إطعام الريا...,0
4,حث أحمد مرتضى منصور، عضو مجلس الزمالك، والمشرف...,0
...,...,...
9995,ترامب يرحب بقرار إف بى آى بالتحقيق فى رسائل إل...,0
9996,حجم الخط ع ع ع قال الدكتور خالد حنفي، وزير الت...,0
9997,وقع انفجار عنيف قرب مركز للشرطة حي يني بوسنة ب...,0
9998,0 57 تحتفظ اليابان بمواقف لمئات الدراجات تحت ا...,0


In [0]:
df = cudf.core.reshape.concat([df_pos, df2], 0)

In [0]:
# Thanks Ayush Kumar
# setting the max length of each article to 200
MAX_LEN = 200
num_sents = df['text'].data.size()

# generate the tokens
seq = df['text'].data.split_record(' ')
# padding each strings if smaller or trim down if larger
for i in range(len(seq)):
  l = seq[i].size()
  if l<= MAX_LEN:
    seq[i] = seq[i].add_strings(nvstrings.to_device((MAX_LEN-l)*['PAD']))
  else:
    seq[i] = seq[i].remove_strings(list(range(MAX_LEN,l)))

In [98]:
print(seq[40])
print(seq[4])
print((len(seq)))

['تعاني', 'الكثيرات', 'مشكلة', 'تساقط', 'الشعر', 'وتقصفه،', 'يكون', 'نتيجة', 'لعوامل', 'عديدة،', 'وخاصة', 'التعرض', 'لأشعة', 'الشمس', 'خلال', 'فصل', 'الصيف', 'لذا', 'يعد', 'الشتاء', 'الفصول', 'المناسبة', 'للعناية', 'والاهتمام', 'بشعرك', 'تقدم', 'فاطمة', 'المرشدي،', 'خبيرة', 'التجميل', 'وصفة', 'طبيعية', 'لتقوية', 'وتنعيم', 'الشعر', 'خلال', '10', 'دقائق', 'فقط', 'الوصفة', 'وتقول', 'فاطمة،', 'قومي', 'بخلط', 'ملعقة', 'كبيرة', 'ماء', 'الورد', 'المركز', 'قطرتين', 'زيت', 'الجاجوبا،', 'وكبسولة', 'فيتامين', 'e', '،', 'ويتم', 'دهن', 'فروة', 'الرأس', 'بالخليط', 'لمدة', 'دقيقة،', 'تركه', 'الشعر', 'لمدة', '10', 'دقائق', 'فقط،', 'ويتم', 'غسل', 'الشعر', 'بالطريقة', 'التقليدية', 'كرري', 'الطريقة', 'مرتين', 'الأسبوع', 'للحصول', 'أفضل', 'النتائج', 'الحفاظ', 'نظام', 'غذائي', 'صحي،', 'وعدم', 'استخدام', 'أدوات', 'الفرد', 'نهائيًا', 'يتعالج', 'الشعر', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

In [99]:
# generating the indices corresponding each token 
c = nvcategory.from_strings_list(seq)
print(c.keys_size())   # total number of unique tokens
print(c.size())       # total number of tokens or vocabulary

153917
2007600


In [100]:
# creating gdf using unique tokens
# TODO more preprocessing, remove \n\n etc...
sent_df = cudf.DataFrame({'tokens':c.keys()})
sent_df.head(10)

,tokens
0,
1,\n\nتعرضت
2,\n\nما
3,\n\nنقد
4,\nارتفع
5,\nتتسم
6,\nيختلف
7,"""المحتفظون"
8,"""بريف"
9,"""بيتكوين"""


In [101]:
# preparing the X_train 
X_train = cuda.device_array((num_sents, MAX_LEN), dtype=np.int32)
c.values(X_train.device_ctypes_pointer.value)
print(X_train.shape)

(10038, 200)


In [102]:
# preparing the y_train
y_train = df['target'].astype('float32').to_gpu_array()
print(y_train.shape)

(10038,)


In [0]:
# creating embedding matrix 
vocab_df = sent_df.merge(pre_df,
                         left_on='tokens',
                         right_on='0',
                         how='left')

In [104]:
#TODO TBFIX Fix multiple nulls
vocab_df.head(60000)

all_token = vocab_df.shape[0]
print(all_token)

153917


In [0]:
import cupy
vocab_df.drop_column('0')
vocab_df.drop_column('tokens')

# filling the not found tokens with random vector
for c in vocab_df.columns:
  vocab_df[c] = vocab_df[c].fillna(cupy.random.normal(size=all_token)).astype(np.float32)

# embedding matrix
vocab = vocab_df.as_gpu_matrix(order='C')

In [0]:
# open issue #23067

def devndarray2tensor(dev_arr, dtyp='float32'):
    dmap = {'float32':torch.float32, 'int32':torch.int32}
    t = torch.empty(size=dev_arr.shape, dtype=dmap[dtyp]).cuda()
    ctx = cuda.cudadrv.driver.driver.get_context()
    
    # constant value of #bytes in float32 = 4
    mp = cuda.cudadrv.driver.MemoryPointer(ctx, ctypes.c_ulong(t.data_ptr()), t.numel()*4)
    tmp_arr = cuda.cudadrv.devicearray.DeviceNDArray(t.size(), [i*4 for i in t.stride()], np.dtype(dtyp), 
                                            gpu_data=mp, stream=torch.cuda.current_stream().cuda_stream)
    tmp_arr.copy_to_device(dev_arr)
    return t

True

In [0]:
def create_emb_layer(weights_matrix, non_trainable=False):
  num_embeddings, embedding_dim = weights_matrix.shape
  emb_layer = nn.Embedding(num_embeddings, embedding_dim)
  emb_layer.weight = nn.Parameter(weights_matrix)
  if non_trainable:
    emb_layer.weight.requires_grad = False
  return emb_layer, num_embeddings, embedding_dim
class ToyLSTM(nn.Module):
  def __init__(self, weights_matrix, hidden_size, output_size, num_layers):
    super(ToyLSTM, self).__init__()
    self.embedding, num_embeddings, embedding_dim = \
                    create_emb_layer(weights_matrix, True)
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,     batch_first=True)
    self.linear = nn.Linear(hidden_size, hidden_size//2)
    self.out = nn.Linear(hidden_size//2, output_size)
    self.relu = nn.ReLU()
  def forward(self, inp):
    h_embedding = self.embedding(inp)
    h_lstm, _ = self.lstm(h_embedding)
    max_pool, _ = torch.max(h_lstm, 1)
    linear = self.relu(self.linear(max_pool))
    out = self.out(linear)
    return out

In [149]:
# instantiate the toy_lstm model
toy_lstm = ToyLSTM(weights_matrix=devndarray2tensor(vocab), hidden_size=50, output_size=1, num_layers=2).cuda()
toy_lstm

ToyLSTM(
  (embedding): Embedding(153917, 300)
  (lstm): LSTM(300, 50, num_layers=2, batch_first=True)
  (linear): Linear(in_features=50, out_features=25, bias=True)
  (out): Linear(in_features=25, out_features=1, bias=True)
  (relu): ReLU()
)

In [150]:
print(X_train.flags)
print(y_train.flags)

{'F_CONTIGUOUS': False, 'C_CONTIGUOUS': True}
{'F_CONTIGUOUS': True, 'C_CONTIGUOUS': True}


In [0]:
split_size = int(0.8 * y_train.shape[0])
train = TensorDataset(devndarray2tensor(X_train[:split_size], dtyp='int32').to(torch.int64), devndarray2tensor(y_train[:split_size]))
trainloader = DataLoader(train, batch_size=256)

valid = TensorDataset(devndarray2tensor(X_train[split_size:], dtyp='int32').to(torch.int64), devndarray2tensor(y_train[split_size:]))
validloader = DataLoader(valid, batch_size=200)

In [0]:
optimizer = optim.Adam(toy_lstm.parameters())
loss_function = nn.BCEWithLogitsLoss(reduction='mean')

In [159]:
for epoch in range(1, 15):
    train_loss, valid_loss = [], []

    # training part
    toy_lstm.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        output = toy_lstm(data)
        loss = loss_function(output, target.view(-1,1))
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        limit = limit - 1
    
    ## evaluation part 
    toy_lstm.eval()
    for data, target in validloader:
        output = toy_lstm(data)
        pred = torch.sum(torch.round(output))
        #print(pred)
        loss = loss_function(output, target.view(-1,1))
        valid_loss.append(loss.item())
valid_loss

[0.003568394109606743,
 0.003659329144284129,
 0.003612331347540021,
 0.0037297499366104603,
 0.003807577071711421,
 0.0038059817161411047,
 0.0035855460446327925,
 0.00360624841414392,
 0.0037001967430114746,
 0.0036332900635898113,
 0.004110497422516346]

In [156]:
train_loss

[0.6500130891799927,
 0.6303161382675171,
 0.6303646564483643,
 0.6302404403686523,
 0.6305034160614014,
 0.6304831504821777,
 0.6305574178695679,
 0.6305416822433472,
 0.6301936507225037,
 0.6301394104957581,
 0.6304129362106323,
 0.6303098201751709,
 0.6305047273635864,
 0.6302539110183716,
 0.6306347846984863,
 0.6301697492599487,
 0.6303194761276245,
 0.6302708983421326,
 0.6302801966667175,
 0.6305937767028809,
 0.6301056742668152,
 0.6302383542060852,
 0.6304757595062256,
 0.6306142807006836,
 0.6302372217178345,
 0.6303231120109558,
 0.6304497718811035,
 0.6305544972419739,
 0.6305256485939026,
 0.6302734613418579,
 0.6304113864898682,
 0.6306394934654236]

In [165]:
from sklearn.metrics import f1_score

dataiter = iter(validloader)
data, labels = dataiter.next()
print(data, labels)
output = toy_lstm(data)
print("out", output, output.shape)

tensor([[ 83204,  81868,  88818,  ...,  97166,   9708,  20359],
        [148241, 119137,  12494,  ...,  84419,  44639, 125203],
        [ 82323,  84519,  83762,  ...,   3039,   3039,   3039],
        ...,
        [ 12298,  19814,  20910,  ...,  49879,  46761, 134078],
        [ 77791, 110287,    481,  ...,   3039,   3039,   3039],
        [ 94111, 110287,  34078,  ...,   3039,   3039,   3039]],
       device='cuda:0') tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.